# Visualizing Stack Overflow Tags Using KeplerMapper
## Table of Contents
 1. [Introduction](#Introduction)
 2. [Implementation](#Implementation)
 3. [Analysis](#Analysis)
 4. [Bonus Proof](#Bonus-Proof)

# Introduction

In the [last project](https://github.com/daniel-rossano/Data-Analysis-Projects/blob/main/Persistent%20Homology%20SO%20Tags/Persistent%20Homology%20of%20Stack%20Overflow%20Tags.ipynb), we looked at a subset of popular [Stack Overflow](https://stackoverflow.com/) tags, and tried to analyze them using persistent homology. Although the previous analysis suggested there were a specific number of "clusters" of tags which are often used together in the same posts, it was lacking because there was no way to determine which tags belong to which clusters, and no way to visualize the clusters.

In this project we attempt to remedy these issues by analyzing tags through a different approach. [KeplerMapper](https://kepler-mapper.scikit-tda.org/en/latest/) is a Python library which implements the Mapper algorithm, a topological data analysis technique that combines dimensionality reduction, clustering, and graph networking to visualize high-dimensional data in a convenient and intuitive way. The idea is that, if we can find a way to represent the SO tags in high-dimensional Euclidean space, we can use [principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA) to reduce dimensionality of the tags, then use KeplerMapper to cluster and visualize the tags. While there are too many technical details to explain here, I will briefly explain the main ideas behind each step of the process as we go through them.

# Implementation

The implementation of this project is surprisingly simple; it is the meaning behind the code that requires more explanation. For our first step, we want to represent the SO tags in Euclidean space. In theory, one could simply represent the tags in two or three dimensions and skip the PCA, but it is difficult to find two or three distinct yet meaningful ways to quantify the SO tags. Instead, we can represent the tags in high-dimensional space by describing tags in terms of their usages and co-occurrences with other tags. For example, if we had only three tags to analyze, we would represent each of them in $\mathbb{R}^3$ by:

$$\text{tag}_1 = (\text{usages of tag}_1, \text{co-occurrences of tag}_1 \text{ and tag}_2, \text{co-occurrences of tag}_1 \text{ and tag}_3)$$
$$\text{tag}_2 = (\text{co-occurrences of tag}_2 \text{ and tag}_1, \text{usages of tag}_2, \text{co-occurrences of tag}_2 \text{ and tag}_3)$$ 
$$\text{tag}_3 = (\text{co-occurrences of tag}_3 \text{ and tag}_1, \text{co-occurrences of tag}_3 \text{ and tag}_2, \text{usages of tag}_3).$$ 

We must therefore restrict this project to a random sample of $50,000$ posts, because SO does not archive co-occurrences of tags across the whole site, so we need to calculate them manually. We restrict to $50,000$ posts  because that is the limit to how many outputs one can get using the [Stack Exchange Data Explorer](https://data.stackexchange.com/stackoverflow/query/new) to obtain this sample.

To make the best usage of this analysis, we will want to filter out a lot of unpopular tags so that they do not cloud up our visualization or waste computation time. Just as we did last time we will start with a sample of $50,000$ random posts to study tags from, and we will only consider tags that have been used in *at least* $100$ posts, i.e., $0.02\%$ of the posts in the sample. As I've learned new things in Python, I've also found a more efficient way to do this than last time. For starters, we no longer need to look at the $50,000$ most popular tags and cross-reference with their usages over the posts. If we query the Data Explorer to report back only the tags used in the random selection of $50,000$ posts, we can just read in the sample using pandas and split the string of tags accordingly.

In [33]:
import pandas as pd
import re
posts = pd.read_csv("SampleofPosts.csv")

all_tags = {} #used to store usages of the tags so it will be easier to sieve out tags used in less than 100 posts
relevant_tags = {} #used to store the usages of only the relevant tags, i.e., only those used in >= 100 posts
count_of_pairs = {} #used to store all co-occurrences of relevant tags


#iterate through each row, and use re.findall with the given pattern to split a given row into a list of tags
for tags in posts["Tags"]:
    tag_list = re.findall(r'<(.*?)>', tags)
    
    #go through each tag in the tag list
    for tag in tag_list:
        if tag in all_tags: #if we've already accounted for the tag in question, increment its usage count
            all_tags[tag] += 1
        else:
            all_tags[tag] = 1 #else, we're storing this tag for the first time



#search through all_tags, and store only the tags used >=100 times in the sample
for tag, count in all_tags.items():
    if count >= 100:
        relevant_tags[tag] = count


#and now we might as well set all_tags to None since we don't need it anymore
all_tags = None

In [34]:
from itertools import combinations #helps us efficiently iterate through combinations of tags

tag_pairs = {} #used to store pairs of tags as keys and their co-occurrences as values

#we'll have to check co-occurrences by reading in tags for each post, just as before
for tags in posts["Tags"]:
    tag_list = re.findall(r'<(.*?)>', tags)
    
    #for all unique pairs of tags in the tag list, if tag1 and tag2 are relevant tags...
    for tag1, tag2 in combinations(tag_list, 2):
        if tag1 in relevant_tags and tag2 in relevant_tags:
            
            #...check if tag1 is lexicographically larger than tag2 to weed out redundant computations
            if tag1 > tag2: 
                tag1, tag2 = tag2, tag1
                
            #and create a tuple of the two tags to act as a key, then proceed as before
            pairing = (tag1, tag2) 
            if pairing in tag_pairs:
                tag_pairs[pairing] += 1
            else:
                tag_pairs[pairing] = 1

Now that we've found and stored all the tag usages and co-occurrences, we can start analyzing the tags. A quick check: 

In [35]:
print(len(relevant_tags.keys()))

187


shows that we're working with $187$ tags, and therefore each tag will be represented as a tuple in $\mathbb{R}^{187}$. Visualizing the tags right now is, of course, not possible for anyone living in less than $187$ dimensions. So naturally the next step is to reduce dimension. For this project, I decided on using PCA because
1. it maximizes variance and is based on linear transformations, so it will be better at preserving structure than say, t-SNE or UMAP
2. it is easy to implement thanks to libraries like sklearn

The basic idea is that we'll run sklearn's `PCA`  on the tag-tuples, reducing it to a three-dimensional space by keeping only the three most important components in each tuple, and then project into this three-dimensional space when we're using KeplerMapper.

To use `PCA`, we will need to use `relevant_tags` and `tag_pairs` to create a 2D numpy array, where each row is represented by one of the tag-tuples. This can be thought of as a $187 \times 187$ matrix, where entry $i,j$ is the usage/co-occurrence of $\text{tag}_i$ with $\text{tag}_j$.

In [36]:
import numpy as np
from sklearn.decomposition import PCA

#use a variable to list all the relevant tags; we'll need this later to label members of clusters, anyway
tag_list = list(relevant_tags.keys())
#initialize the 2D array as n x n
tag_matrix = np.zeros((len(tag_list), len(tag_list)))

#this will be much less painful if we keep track of the indices of the matrix
#so we will create a simple mapping of tags to sorted indices in the tag list
index_map = {tag: index for index, tag in enumerate(tag_list)}

#now we can easily fill in the diagonals
for tag, count in relevant_tags.items():
    i = index_map[tag] #find the index of the tag so we put the count in the right entry
    tag_matrix[i,i] = count

#fill in the non-diagonal entries analogously, keeping in mind this is a symmetric matrix
for (tag_A, tag_B), count in tag_pairs.items():
    i = index_map[tag_A]
    j = index_map[tag_B]
    tag_matrix[i,j] = count
    tag_matrix[j,i] = count

#no need for index_map anymore
index_map = None

#initialize sklearn's PCA, remembering to keep 3 principal components
pca = PCA(n_components = 3)
#now fit_transform the tag matrix and store the resulting array
tag_pca = pca.fit_transform(tag_matrix)

One last thing we must consider is our choice of clustering algorithm. The Mapper algorithm works by projecting a high-dimensional data set into a low-dimensional data set, choosing a cover for the low-dimensional data set (KM only allows for cubes at this point in time), and then clustering points locally based on their location in the cover. Next the Mapper algorithm constructs a graph where each node represents a cluster, and whenever a point lies inside more than one cluster (which may happen due to overlap among cubes), an edge will be drawn between the corresponding nodes. For the record, KM allows for *any* projection function, and *any* clustering algorithm or distance metric. I chose to use k-means clustering for this as that seems to be the standard for text clustering. We can again rely on sklearn to provide us with an algorithm for k-means clustering. 

We will initialize the graph formed by the Mapper algorithm in one line. Then, we will want to make sure we're able to track which tags belong to which clusters, so we will need to form a numpy array of all the tags and use it for the `custom_tooltips` parameter to ensure KM labels the tags appropriately. We will also separate the tags by square brackets so they are easier to read on the graph. Finally, we will use `KeplerMapper.visualize` to output a .html file of the graph and begin the analysis. Because this project is written in Jupyter Notebook, we must take the extra step of importing KM's `jupyter` to create a custom display of the .html file using the notebook.

In [37]:
import sklearn
import kmapper as km
from IPython.display import IFrame #used to display the .html in the notebook environment

#initialize mapper
mapper = km.KeplerMapper()
#and now build the graph; cluster using specified # of kmeans clusters; build cover w/ % overlap
graph = mapper.map(tag_pca, tag_matrix, clusterer = sklearn.cluster.KMeans(n_clusters = 6), 
                   cover = km.Cover(n_cubes = 20, perc_overlap = 0.3))

#to construct the array just modify the existing tag list and convert to np array
#the conditional is here to prevent adding multiple sets of brackets upon re-running this cell
if tag_list[0].find('[') == -1:
    tag_list = ['[' + tag + ']' for tag in tag_list]
labels = np.array(tag_list)
#finally, we visualize and output the graph
mapper.visualize(graph, path_html="Visualization-of-SO-Tags.html", title = 'Clusters of Stack Overflow Tags',
                custom_tooltips = labels)

IFrame(src = "Visualization-of-SO-Tags.html", width=800, height=600)

# Analysis

We've obtained an html file, and now all that remains is to ask, "what does it tell us about the Stack Overflow tags?" For starters, one should keep in mind that there are *many* different ways to change the simplicial complex generated by KM: projection function, filtering function and its number of clusters or definition of closeness, number of cubes, and percentage of cube overlap can all dramatically change the output with even the slightest adjustment. The choices of projection and filtering function were already justified, though they may not be perfect! As for why we used $6$ k-means clusters and $20$ cubes with $30\%$ overlap, it was essentially the result of trial-and-error, with these parameters yielding the most interesting results.

In short, I believe that the current output does suggest a few relationships among SO tags, however there are likely other methods of analyzing these tags. Unfortunately there are some "mega nodes" which contain such a vast majority of tags that they cannot possibly be described by one concept, programming language, or computer science discipline. Attempts to "break up" these clusters into smaller, more descriptive clusters by changing the parameters mentioned above did not seem any more productive. Additionally, there are tags that one would expect to be incredibly popular, like "C++," that appear to be isolated from every other tag, which suggests not all important features of the data were captured in this process. It is possible that PCA may not be the optimal method of dimensionality reduction for this data set, or perhaps that another, non-linear clustering algorithm like DBSCAN or t-SNE may be useful for further analysis.

On a more positive note, in between the "isolated nodes" and "mega nodes," there are some smaller nodes having $2-25$ members with commonalities. Of particular interest may be the "web-dev node" of $25$ members, containing tags such as "xml," "sqlite," and "android-studio," as the majority of its entries refer directly to technologies, environments, and protocols used in web development. This finding seems to suggest that the work done here is not in vain, but rather in need of further exploration.


# Bonus Proof
This part is mathematically rigorous in nature, and not essential to the project itself. But since I enjoyed thinking about this so much, I wanted to include it here! One may wonder how well the analysis of tags using KeplerMapper holds up against the analysis using persistent homology (PH). Although the overall number of clusters differs quite a bit from the PH approach, this is mostly due to the choice of deciding which clusters counted as sufficiently persistent. In theory, both approaches are actually quite similar; understanding why requires some mathematical understanding of what is happening in each project.

For the PH approach, we clustered tags by defining an abstract finite metric space $M = (X,d)$, where $X$ is a set whose elements are the set $P$ of all posts in the sample which are tagged with some tag $T$, and $d$ is the Jaccard metric, defined by $d(P,Q) = 1 - \frac{|P \cap Q|}{|P \cup Q|}$ (here $| \cdot |$ denotes the cardinality of a set).

In the analysis using KeplerMapper, we've embedded the tags into $\mathbb{R}^n$ for $n = |X|$, using a finite set $Y$, whose elements are tuples are of the form $y_i = (|X_i \cap X_1|, |X_i \cap X_2|, \ldots , |X_i \cap X_n|)$, where $X_i$ is a fixed element of $X$ and $X_1,X_2,\ldots,X_n$ are the $n$ elements of $X$. Any clustering algorithms are applying the Euclidean metric $| \cdot |$ on $Y$, and therefore the difference between the two projects is that we've gone from working with the metric space $M$ to working with the metric space $N = (Y, | \cdot |)$. In particular, this means our results will be similar if there is some way to preserve the notion of "closeness" between elements of $M$ and elements of $N$. In other (topological) words, the results are the same if there exists a continuous function $f:M \to N$.

The following proof constructs such a function $f$ by formalizing the ideas in this project. We show, in fact, that our choice $f$ is uniformly continuous.

**Lemma:**

Let $X = \{X_i\}^{i=n} _{i=1}$ be a finite collection of nonempty finite sets, and let $M = (X, d)$ be the finite metric space formed by equipping $X$ with the metric $d(X_i, X_j) = 1 - \frac{|X_i \cap X_j|}{|X_i \cup X_j|}$. Now let $Y \subset \mathbb{R}^n$ be a finite set consisting of all tuples of the form $y_i = (|X_i \cap X_1|, |X_i \cap X_2|, \ldots , |X_i \cap X_n|)$ for each $1 \leq i \leq n$. If $N = (Y, | \cdot |)$, then there exists a uniformly continuous function $f:M \to N$.

*proof:*

Define $f:M \to N$ by $f(X_i) = y_i$. Because $y_i$ is defined by $X_i$, we know $f$ is well-defined (and one can also see that $f$ is bijective since it is a surjection between finite sets of the same cardinality). For any $\epsilon > 0$, choose $\delta = \min \{d(X_i, X_j) | X_i \neq X_j\}$. We know such a $\delta$ exists because $X$ is finite, hence there are only finitely-many pairs of distances of $X_i \neq X_j$, and therefore one of these distances must be the strictly positive smallest.

Then for any $X_i, X_j \in X$ such that $d(X_i, X_j) < \delta$, by our choice of $\delta$ it must be true that $X_i = X_j$ so that $d(X_i, X_j) = 0 < \delta$. Importantly, $X_i = X_j$ means that $y_i = y_j$, since $y_i = (|X_i \cap X_1|, \ldots , |X_i \cap X_n|) = (|X_j \cap X_1|, \ldots , |X_j \cap X_n|) = y_j$. Then $|f(X_i) - f(X_j)| = |y_i - y_j| = 0 < \epsilon$ whenever $d(X_i, X_j) < \delta$. Since our choice of $\delta$ was independent of any point in $X$, this shows that $f$ is uniformly continuous.

Thus we have shown that distances between points in these two metric spaces can be preserved, and consequently our methods of analysis using PH is not nonsensical compared to our methods of analysis using KeplerMapper.